<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


# Document Clustering - Opinion Review 데이터 셋

data source : https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

## data load

위의 UCI 에서 topics 에  txt 파일을 csv로 변환 해둠


[변환 githup](https://github.com/SeWonKwon/ML_training/blob/main/UCI/Opinosis_review/01_data%20txt%20to%20csv.ipynb)

In [1]:
import pandas as pd
document_df = pd.read_csv('https://raw.githubusercontent.com/SeWonKwon/ML_training/main/UCI/Opinosis_review/data/topics.csv', index_col=0)

In [2]:
document_df.head(3)

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...


In [3]:
document_df['opinion_text'] = document_df['opinion_text'].str.strip()
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 ..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ver..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my comp...
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ..."


# tokenize & Lemmatize

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1, 2), min_df=0.05, max_df=0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

C:\Users\bigne\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


# Kmeans Clustering

by 5

In [6]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)


KMeans(max_iter=10000, n_clusters=5, random_state=0)

In [7]:
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [8]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 ...",2
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ver...",0
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my comp...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",1


In [9]:
document_df.value_counts('cluster_label').sort_index()

cluster_label
0     4
1     8
2    16
3    13
4    10
dtype: int64

In [10]:
for i, j in document_df.groupby('cluster_label'):
    print('cluster {}'.format(i))
    print(j['filename'],'\n')
    

cluster 0
1     bathroom_bestwestern_hotel_sfo
30       rooms_bestwestern_hotel_sfo
31           rooms_swissotel_chicago
32           room_holiday_inn_london
Name: filename, dtype: object 

cluster 1
2      battery-life_amazon_kindle
3      battery-life_ipod_nano_8gb
4     battery-life_netbook_1005ha
19        keyboard_netbook_1005ha
26     performance_netbook_1005ha
41       size_asus_netbook_1005ha
42            sound_ipod_nano_8gb
44                 speed_windows7
Name: filename, dtype: object 

cluster 2
0       accuracy_garmin_nuvi_255W_gps
5               buttons_amazon_kindle
8     directions_garmin_nuvi_255W_gps
9        display_garmin_nuvi_255W_gps
10      eyesight-issues_amazon_kindle
11                  features_windows7
12                fonts_amazon_kindle
23           navigation_amazon_kindle
33     satellite_garmin_nuvi_255W_gps
34        screen_garmin_nuvi_255W_gps
35               screen_ipod_nano_8gb
36              screen_netbook_1005ha
43         speed_garmin_nuvi_2

by 3

In [11]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)


KMeans(max_iter=10000, n_clusters=3, random_state=0)

In [12]:
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [13]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 ...",0
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ver...",2
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my comp...,0
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n...,0
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",0


In [14]:
document_df.value_counts('cluster_label').sort_index()

cluster_label
0    25
1    10
2    16
dtype: int64

In [15]:
for i, j in document_df.groupby('cluster_label'):
    print('cluster {}'.format(i))
    print(j['filename'],'\n')
    

cluster 0
0       accuracy_garmin_nuvi_255W_gps
2          battery-life_amazon_kindle
3          battery-life_ipod_nano_8gb
4         battery-life_netbook_1005ha
5               buttons_amazon_kindle
8     directions_garmin_nuvi_255W_gps
9        display_garmin_nuvi_255W_gps
10      eyesight-issues_amazon_kindle
11                  features_windows7
12                fonts_amazon_kindle
19            keyboard_netbook_1005ha
23           navigation_amazon_kindle
26         performance_netbook_1005ha
27                price_amazon_kindle
33     satellite_garmin_nuvi_255W_gps
34        screen_garmin_nuvi_255W_gps
35               screen_ipod_nano_8gb
36              screen_netbook_1005ha
41           size_asus_netbook_1005ha
42                sound_ipod_nano_8gb
43         speed_garmin_nuvi_255W_gps
44                     speed_windows7
48       updates_garmin_nuvi_255W_gps
49                video_ipod_nano_8gb
50         voice_garmin_nuvi_255W_gps
Name: filename, dtype: object 

cluster 

전자기기, 호텔, 자동차 3개로 잘 분류 된것을 볼수 있다. 

# 군집별 핵심 단어 추출하기

In [16]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [17]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [18]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')


In [19]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
####### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_h

**Reference**

* <a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

<br>

* 파이썬 머신러닝 완벽 가이드, 권철민
* [이수안 컴퓨터 연구소](http://suanlab.com/)